### Задание 1. Базовое изучение

Изучить представленный набор данных на основе [описания его столбцов](https://raw.githubusercontent.com/obulygin/pyda_homeworks/master/statistics_basics/horse_data.names) и выбрать 8 столбцов для дальнейшего изучения (среди них должны быть как числовые, так и категориальные). Провести расчет базовых метрик для них, кратко описать результаты.

In [256]:
import pandas as pd
import numpy as np

In [257]:
horse = pd.read_csv('c:/ds22/1_pyda/15_statistic/horse_data.csv', sep=',',header=None, na_values='?')
horse.head()

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,2.0,1,530101,38.5,66.0,28.0,3.0,3.0,NaN,2.0,...,45.0,8.4,NaN,NaN,2.0,2,11300,0,0,2
1,1.0,1,534817,39.2,88.0,20.0,NaN,NaN,4.0,1.0,...,50.0,85.0,2.0,2.0,3.0,2,2208,0,0,2
2,2.0,1,530334,38.3,40.0,24.0,1.0,1.0,3.0,1.0,...,33.0,6.7,NaN,NaN,1.0,2,0,0,0,1
3,1.0,9,5290409,39.1,164.0,84.0,4.0,1.0,6.0,2.0,...,48.0,7.2,3.0,5.3,2.0,1,2208,0,0,1
4,2.0,1,530255,37.3,104.0,35.0,NaN,NaN,6.0,2.0,...,74.0,7.4,NaN,NaN,2.0,2,4300,0,0,2


In [258]:
# Выбираем восемь столбцов
df = horse[[0,  1,  2,  3,  4,  5,  6, 18]]
df. columns = ['surgery','age','number','rectemp','pulse','resprate','tempofextr','packcellvolume']
df.head()

,surgery,age,number,rectemp,pulse,resprate,tempofextr,packcellvolume
0,2.0,1,530101,38.5,66.0,28.0,3.0,45.0
1,1.0,1,534817,39.2,88.0,20.0,NaN,50.0
2,2.0,1,530334,38.3,40.0,24.0,1.0,33.0
3,1.0,9,5290409,39.1,164.0,84.0,4.0,48.0
4,2.0,1,530255,37.3,104.0,35.0,NaN,74.0


#### Типы данных в выбранных столбцах.
##### Категориальные:
- surgery - (1 = it had surgery, 2 = It was treated without surgery);
- age - (1 = Adult horse, 2 = Young (< 6 months));
- number - the case number assigned to the horse;
- tempofextr - temperature of extremities (1 = Normal, 2 = Warm, 3 = Cool, 4 = Cold).

##### Числовые:
- rectemp - temperature in degrees celsius, normal temp is 37.8;
- pulse - the heart rate in beats per minute, 30-40 is normal for adults;
- resprate - respiratory rate, normal rate is 8 to 10
- packcellvolume - the # of red cells by volume in the blood, normal range is 30 to 50.

Для количественных столбцов выведем основные статистические метрики.
Видим, что значения имеют большой разброс, и средние величины
сильно отличаются от нормы, но, возможно, у больных лошадей такое может быть.

In [259]:
df[['rectemp','pulse','resprate','packcellvolume']].describe()

,rectemp,pulse,resprate,packcellvolume
count,240.000000,276.000000,242.000000,271.000000
mean,38.167917,71.913043,30.417355,46.295203
std,0.732289,28.630557,17.642231,10.419335
min,35.400000,30.000000,8.000000,23.000000
25%,37.800000,48.000000,18.500000,38.000000
50%,38.200000,64.000000,24.500000,45.000000
75%,38.500000,88.000000,36.000000,52.000000
max,40.800000,184.000000,96.000000,75.000000


In [260]:
# Дополнительно расчитаем моду
print('mode rectemp')
print(df.rectemp.round().mode()[0])
print('mode pulse')
print(df.pulse.round().mode()[0])
print('mode resprate')
print(df.resprate.round().mode()[0])
print('mode packcellvolume')
print(df.packcellvolume.round().mode()[0])

mode rectemp
38.0
mode pulse
48.0
mode resprate
20.0
mode packcellvolume
37.0


Для категориальных переменных возможные значения категорий указаны в описании, 
поэтому посмотрим количество строк в каждой категории (кроме number, которое является id лошади).

In [261]:
print(df['surgery'].value_counts())
print(df['age'].value_counts())
print(df['tempofextr'].value_counts())

1.0    180
2.0    119
Name: surgery, dtype: int64
1    276
9     24
Name: age, dtype: int64
3.0    109
1.0     78
2.0     30
4.0     27
Name: tempofextr, dtype: int64


Для surgery и tempofextr в датасете представлены значения, которые указаны в описании.  
В столбце age вместо значения 2 = Young, есть значение 9. Возможно, это ошибка описания.

### Задание 2. Работа с выбросами
В выбранных числовых столбцах найти выбросы, выдвинуть гипотезы об их причинах и проинтерпретировать результаты. Принять и обосновать решение о дальнейшей работе с ними.

#### Выбросы будем определять только для количественных переменных. 
- Выбросы параметра rectemp

In [262]:
q1 = df.rectemp.quantile(0.25)
q3 = df.rectemp.quantile(0.75)
iqr = q3-q1
lower_bond = q1 - (1.5 * iqr)
upper_bond = q3 + (1.5 * iqr)
print(f'rectemp lower_bond = {lower_bond:.2f}')
print(f'rectemp upper_bond = {upper_bond:.2f}')
df[(df.rectemp < lower_bond) | (df.rectemp > upper_bond)].sort_values('rectemp')

rectemp lower_bond = 36.75
rectemp upper_bond = 39.55


,surgery,age,number,rectemp,pulse,resprate,tempofextr,packcellvolume
44,1.0,1,535407,35.4,140.0,24.0,3.0,57.0
141,2.0,1,522979,36.0,42.0,30.0,NaN,64.0
238,2.0,1,528702,36.1,88.0,NaN,3.0,45.0
80,1.0,1,527518,36.4,98.0,35.0,3.0,47.0
118,1.0,1,533983,36.5,78.0,30.0,1.0,34.0
298,1.0,1,530612,36.5,100.0,24.0,3.0,50.0
251,2.0,1,527940,36.6,42.0,18.0,3.0,52.0
99,2.0,1,530002,39.6,108.0,51.0,3.0,59.0
75,1.0,9,534092,39.7,100.0,NaN,3.0,48.0
20,1.0,1,530157,39.9,72.0,60.0,1.0,46.0


Так как речь идет о медицинских показателях, то правильнее выбросами считать значения, которые невозможны с точки зрения медицины. 
С этой точки зрения вызывают вопросы значения, которые считаются выбросами при расчете с помощью межквартильного интервала. Поэтому, прежде чем решить как поступать с выбросами я проверила в открытых источниках, возможны ли такие значения.  
Например, я нашла информацию, что температура может упасть или подняться на 2-4 градуса, то есть значения за пределами границ определенных при помощи межквартильного интервала реальны, и данные выбросы мы исключать не будем.

- Выбросы параметра pulse  
Так как минимальное значение пульса равно 30, что является нормальным значением, будем определять только верхнюю границу по межквартильному интервалу. 
Так как пульс лошади может подняться до 180-240 ударов, значения выше верхней границы не будем исключать.

In [263]:
q1 = df.pulse.quantile(0.25)
q3 = df.pulse.quantile(0.75)
iqr = q3-q1
upper_bond = q3 + (1.5 * iqr)
print(f'pulse upper_bond = {upper_bond}')
df[(df.pulse > upper_bond)]

pulse upper_bond = 148.0


,surgery,age,number,rectemp,pulse,resprate,tempofextr,packcellvolume
3,1.0,9,5290409,39.1,164.0,84.0,4.0,48.0
41,2.0,9,5288249,39.0,150.0,72.0,NaN,47.0
55,1.0,9,5282839,38.6,160.0,20.0,3.0,38.0
255,1.0,9,5294539,38.8,184.0,84.0,1.0,33.0
275,1.0,9,5297159,38.8,150.0,50.0,1.0,50.0


- Выбросы параметра resprate   
Так как минимальное значение частоты дыхания равно 8, что является нормальным значением, будем определять только верхнюю границу по межквартильному интервалу. 
Так как частота дыхания у жеребят может быть до 130, значения выше верхней границы не будем исключать.

In [264]:
q1 = df.resprate.quantile(0.25)
q3 = df.resprate.quantile(0.75)
iqr = q3-q1
upper_bond = q3 + (1.5 * iqr)
print(f'resprate upper_bond = {upper_bond}')

resprate upper_bond = 62.25


- Выбросы параметра packcellvolume  
При норме от 30 до 50 разброс значений показателя в пределах от 23 до 75 вполне возможен. По этой причине исключать выбросы, определенные при помощи меквартиьного интервала исключать не будем.

In [265]:
q1 = df.packcellvolume.quantile(0.25)
q3 = df.packcellvolume.quantile(0.75)
iqr = q3-q1
lower_bond = q1 - (1.5 * iqr)
upper_bond = q3 + (1.5 * iqr)
print(f'packcellvolume lower_bond = {lower_bond:.2f}')
print(f'packcellvolume upper_bond = {upper_bond:.2f}')
df[(df.packcellvolume < lower_bond) | (df.packcellvolume > upper_bond)].sort_values('packcellvolume')

packcellvolume lower_bond = 17.00
packcellvolume upper_bond = 73.00


,surgery,age,number,rectemp,pulse,resprate,tempofextr,packcellvolume
4,2.0,1,530255,37.3,104.0,35.0,NaN,74.0
144,1.0,1,534519,37.1,84.0,40.0,3.0,75.0
151,1.0,1,529399,39.3,NaN,NaN,4.0,75.0


### Задание 3. Работа с пропусками
Рассчитать количество выбросов для всех выбранных столбцов. Принять и обосновать решение о методе работы с пропусками по каждому столбцу, сформировать датафрейм, в котором пропуски будут отсутствовать.

In [266]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   surgery         299 non-null    float64
 1   age             300 non-null    int64  
 2   number          300 non-null    int64  
 3   rectemp         240 non-null    float64
 4   pulse           276 non-null    float64
 5   resprate        242 non-null    float64
 6   tempofextr      244 non-null    float64
 7   packcellvolume  271 non-null    float64
dtypes: float64(6), int64(2)
memory usage: 18.9 KB


Анализ датасета показывает, что пропущенные значения есть во всех столбцах, кроме age и number. 

In [267]:
for col in df.columns:
    missing = df[col].isnull().mean()
    print(f'{col} - {missing:.1%}')

surgery - 0.3%
age - 0.0%
number - 0.0%
rectemp - 20.0%
pulse - 8.0%
resprate - 19.3%
tempofextr - 18.7%
packcellvolume - 9.7%


Так как процент пропущенных значений не превышает 20%, столбцы удалять не будем.  
Для работы с пропущенными значениями предлагается следующий поход:
- найти строки, в которых пропущены значения во всех столбцах кроме surgery, age и number и удалить их;
- разделить датасет на четыре группы: по возрасту и наличию операции и заполнить пропущенные значения средними значениями для каждой группы;
- при заполнении для столбцов с категориальными величинами (surgery и tempofextr) использовать моду, для столбцов с количественными величинами (rectemp, pulse, resprate и packcellvolume) использовать медиану.

In [268]:
df[df.rectemp.isnull() & df.pulse.isnull() & df.resprate.isnull() & df.tempofextr.isnull() & df.packcellvolume.isnull()]

,surgery,age,number,rectemp,pulse,resprate,tempofextr,packcellvolume
28,1.0,1,5279442,NaN,NaN,NaN,NaN,NaN
175,1.0,1,527929,NaN,NaN,NaN,NaN,NaN
216,2.0,1,528977,NaN,NaN,NaN,NaN,NaN
288,1.0,1,529428,NaN,NaN,NaN,NaN,NaN


In [269]:
drop_list = df.index[df.rectemp.isnull() & df.pulse.isnull() & df.resprate.isnull() & df.tempofextr.isnull() & df.packcellvolume.isnull()]
print(drop_list)
#df.drop(drop_list, inplace=True)
df = df.drop(drop_list)
len(df)

Int64Index([28, 175, 216, 288], dtype='int64')


296

#### Пропуски в столбце surgery

In [270]:
print(df.surgery.isna().sum())
df[df.surgery.isnull()]

1


,surgery,age,number,rectemp,pulse,resprate,tempofextr,packcellvolume
132,NaN,1,534572,38.0,48.0,20.0,3.0,43.0


In [271]:
# ПРоверим, не ли другого значения для этого id
df[df.number == 534572]

,surgery,age,number,rectemp,pulse,resprate,tempofextr,packcellvolume
132,NaN,1,534572,38.0,48.0,20.0,3.0,43.0


In [272]:
# Так как другой строки с таким id нет, заменим пропущенное значение модой для категории age = 1
print(df[df.age==1].surgery.mode()[0])
df.surgery.fillna(df[df.age==1].surgery.mode()[0], inplace=True)
print(df.surgery.isna().sum())

1.0
0


В остальных столбцах будем заполнять медианой на основе группировки по столбцам surgery и age

#### Пропуски в столбце rectemp

In [273]:
print(df.groupby(['surgery','age']).rectemp.median())
df.rectemp.fillna(df.groupby(['surgery','age']).rectemp.transform('median'), inplace=True)
print(df.groupby(['surgery','age']).rectemp.median())

surgery  age
1.0      1      38.1
         9      38.6
2.0      1      38.2
         9      38.6
Name: rectemp, dtype: float64
surgery  age
1.0      1      38.1
         9      38.6
2.0      1      38.2
         9      38.6
Name: rectemp, dtype: float64


#### Пропуски в столбце pulse

In [278]:
print(df.groupby(['surgery','age']).pulse.median())
df.pulse.fillna(df.groupby(['surgery','age']).pulse.transform('median'), inplace=True)
print(df.groupby(['surgery','age']).pulse.median())

surgery  age
1.0      1       66.0
         9      129.0
2.0      1       54.0
         9       91.0
Name: pulse, dtype: float64
surgery  age
1.0      1       66.0
         9      129.0
2.0      1       54.0
         9       91.0
Name: pulse, dtype: float64


#### Пропуски в столбце resprate

In [274]:
print(df.groupby(['surgery','age']).resprate.median())
df.resprate.fillna(df.groupby(['surgery','age']).resprate.transform('median'), inplace=True)
print(df.groupby(['surgery','age']).resprate.median())

surgery  age
1.0      1      24.0
         9      60.0
2.0      1      24.0
         9      30.0
Name: resprate, dtype: float64
surgery  age
1.0      1      24.0
         9      60.0
2.0      1      24.0
         9      30.0
Name: resprate, dtype: float64


#### Пропуски в столбце tempofextr

In [275]:
print(df.groupby(['surgery','age']).tempofextr.median())
df.tempofextr.fillna(df.groupby(['surgery','age']).tempofextr.transform('median'), inplace=True)
print(df.groupby(['surgery','age']).tempofextr.median())

surgery  age
1.0      1      3.0
         9      3.0
2.0      1      2.0
         9      1.0
Name: tempofextr, dtype: float64
surgery  age
1.0      1      3.0
         9      3.0
2.0      1      2.0
         9      1.0
Name: tempofextr, dtype: float64


#### Пропуски в столбце packcellvolume


In [276]:
print(df.groupby(['surgery','age']).packcellvolume.median())
df.packcellvolume.fillna(df.groupby(['surgery','age']).packcellvolume.transform('median'), inplace=True)
print(df.groupby(['surgery','age']).packcellvolume.median())

surgery  age
1.0      1      45.0
         9      45.0
2.0      1      44.0
         9      37.0
Name: packcellvolume, dtype: float64
surgery  age
1.0      1      45.0
         9      45.0
2.0      1      44.0
         9      37.0
Name: packcellvolume, dtype: float64


In [279]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 296 entries, 0 to 299
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   surgery         296 non-null    float64
 1   age             296 non-null    int64  
 2   number          296 non-null    int64  
 3   rectemp         296 non-null    float64
 4   pulse           296 non-null    float64
 5   resprate        296 non-null    float64
 6   tempofextr      296 non-null    float64
 7   packcellvolume  296 non-null    float64
dtypes: float64(6), int64(2)
memory usage: 20.8 KB


#### Итог: датасет не содержит пропущенных значений